## Evaluation for RAG

In [1]:
## RAG
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# List of URLs to load documents from
urls = [
    'https://lilianweng.github.io/posts/2023-06-23-agent/',
    'https://lilianweng.github.io/posts/2024-04-12-diffusion-video/',
    'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/',
    'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering',
]

/home/crizz/python/rag_bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
# Load documents from the URL
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
chunks = splitter.split_documents(docs_list)

# Vectorstore
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(embedding=embeddings, documents=chunks)

# Retriever
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [8]:
retriever.invoke("What is agents?")

[Document(id='0ab07190-9fb7-455b-a97a-50088d194fc1', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, 

In [9]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen3:4b")

In [10]:
from langsmith import traceable

## Add decorator
@traceable
def rag_bot(question: str) -> dict:
    ## Relevant context
    docs = retriever.invoke(question)
    docs_string = " ".join(doc.page_content for doc in docs)
    
    # Prompt
    instructions = f"""You are a helpful assistant who is good at analyzing source information answer the user's questions.
    If you don't know the anser, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    
    Documents:
    {docs_string}
    """
    
    # LLM
    result = llm.invoke([
        {'role':'system', 'content': instructions},
        {'role': 'user', 'content': question}
    ])
    
    return {"answer": result.content, "documents": docs}
    

In [11]:
rag_bot("What is agents?")

{'answer': 'Based on the provided documents, generative agents are virtual characters controlled by LLMs that interact in a sandbox environment inspired by The Sims, forming believable relationships and planning reactions based on observations. The system evaluates their tool use capabilities through planning, memory, and specific case studies like scientific discovery.',
 'documents': [Document(id='0ab07190-9fb7-455b-a97a-50088d194fc1', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent syste

In [ ]:
### Dataset
from langsmith import Client

client = Client()
